In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys, BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.enums import PositionModeType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.indicators.tv_indicators import *
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})


%load_ext autoreload
%autoreload 2

In [2]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)
mufex_candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)

mufex_open = mufex_candles[:, 1]
mufex_high = mufex_candles[:, 2]
mufex_low = mufex_candles[:, 3]
mufex_close = mufex_candles[:, 4]


mufex_close_shift = np.roll(mufex_candles[:, 4], 1)
mufex_close_shift[0] = np.nan
mufex_close

array([37111.2, 37111.1, 37130.9, 37125.7, 37124.1, 37114.4, 37110.8,
       37122.6, 37116.8, 37080.2, 37122.4, 37123.7, 37089.4, 37065.2,
       37060.8, 37063.9, 37101.3, 37090.8, 37070.7, 37015.2, 36964.0,
       37011.6, 37085.0, 37063.6, 37045.4, 37093.3, 37088.0, 37105.0,
       37104.4, 37119.2, 37105.2, 37149.7, 37145.2, 37143.3, 37133.2,
       37095.3, 37129.3, 37110.9, 37098.1, 37082.0, 37058.0, 37046.5,
       37030.1, 37069.1, 37091.8, 37094.1, 37118.6, 37145.7, 37125.0,
       37121.8, 37115.3, 37118.9, 37088.9, 37077.0, 37055.1, 37080.6,
       37090.3, 37114.1, 37079.1, 37043.2, 36987.3, 36975.4, 37030.5,
       36997.1, 36988.3, 36984.7, 37022.9, 37023.5, 37040.3, 37009.9,
       37014.5, 36987.2, 37000.4, 36984.3, 36989.8, 36981.6, 36968.2,
       36981.9, 37044.0, 37079.0, 37073.7, 37067.9, 37024.1, 37050.0,
       37020.6, 37050.8, 37058.6, 37041.0, 37039.6, 37030.0, 37031.8,
       37019.7, 36981.6, 37003.7, 37002.2, 37015.7, 36987.4, 37019.8,
       37036.7, 3700

In [3]:
mufex_main.create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="limit",
    price=30000,
    asset_size=0.001,
    time_in_force="GoodTillCancel",
)

'6dfafcde-451a-4cc6-8c8f-b25b8502d909'

In [4]:
dl_ex_candles(
    exchange="apex",
    symbol="BTCUSDC",
    timeframe="5m",
    candles_to_dl=200,
)

array([[1699670700000.0, 37107.0, 37122.0, 37106.5, 37121.5],
       [1699671000000.0, 37121.5, 37156.5, 37121.5, 37142.5],
       [1699671300000.0, 37142.5, 37166.0, 37142.5, 37160.5],
       [1699671600000.0, 37160.5, 37173.0, 37130.5, 37154.5],
       [1699671900000.0, 37154.5, 37156.0, 37104.0, 37150.0],
       [1699672200000.0, 37150.0, 37154.5, 37108.0, 37147.0],
       [1699672500000.0, 37147.0, 37154.0, 37114.0, 37139.0],
       [1699672800000.0, 37139.0, 37161.0, 37134.0, 37155.5],
       [1699673100000.0, 37155.5, 37169.0, 37141.0, 37148.0],
       [1699673400000.0, 37148.0, 37162.0, 37106.0, 37111.5],
       [1699673700000.0, 37111.5, 37155.5, 37102.0, 37155.5],
       [1699674000000.0, 37155.5, 37175.0, 37142.0, 37153.0],
       [1699674300000.0, 37153.0, 37155.5, 37109.0, 37109.0],
       [1699674600000.0, 37109.0, 37121.5, 37086.0, 37086.0],
       [1699674900000.0, 37086.0, 37105.0, 37077.0, 37089.5],
       [1699675200000.0, 37089.5, 37100.5, 37055.5, 37088.5],
       [

In [5]:
apex_test = Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True
)
apex_test.create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="sell",
    asset_size=0.005,
)

'511261118828642924'

In [6]:
b_candles = dl_ex_candles(
    exchange="binance_usdm",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)
b_candles

array([[1699670700000.0, 37077.9, 37119.0, 37076.0, 37111.1],
       [1699671000000.0, 37111.1, 37126.6, 37104.3, 37111.5],
       [1699671300000.0, 37111.5, 37139.6, 37111.5, 37131.1],
       [1699671600000.0, 37131.1, 37147.9, 37104.8, 37125.9],
       [1699671900000.0, 37125.9, 37130.0, 37078.8, 37124.5],
       [1699672200000.0, 37124.5, 37125.5, 37082.6, 37114.4],
       [1699672500000.0, 37114.5, 37122.4, 37084.4, 37110.5],
       [1699672800000.0, 37110.5, 37132.7, 37108.1, 37122.2],
       [1699673100000.0, 37122.3, 37135.5, 37109.5, 37117.0],
       [1699673400000.0, 37117.0, 37130.0, 37076.9, 37080.1],
       [1699673700000.0, 37080.1, 37124.4, 37061.5, 37121.3],
       [1699674000000.0, 37121.2, 37146.0, 37107.2, 37121.0],
       [1699674300000.0, 37121.0, 37128.0, 37081.3, 37089.5],
       [1699674600000.0, 37089.6, 37094.4, 37065.0, 37065.1],
       [1699674900000.0, 37065.1, 37080.0, 37055.6, 37064.2],
       [1699675200000.0, 37064.2, 37073.1, 37029.4, 37064.3],
       [

In [7]:
candles_to_df(candles=b_candles)

,timestamp,open,high,low,close
datetime,,,,,
2023-11-11 02:45:00,1699670700000,37077.9,37119.0,37076.0,37111.1
2023-11-11 02:50:00,1699671000000,37111.1,37126.6,37104.3,37111.5
2023-11-11 02:55:00,1699671300000,37111.5,37139.6,37111.5,37131.1
2023-11-11 03:00:00,1699671600000,37131.1,37147.9,37104.8,37125.9
2023-11-11 03:05:00,1699671900000,37125.9,37130.0,37078.8,37124.5
...,...,...,...,...,...
2023-11-11 19:00:00,1699729200000,37140.7,37143.9,37121.4,37143.9
2023-11-11 19:05:00,1699729500000,37143.9,37143.9,37126.5,37136.0
2023-11-11 19:10:00,1699729800000,37135.9,37136.0,37112.0,37112.1


In [8]:
binance_test = BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
)
binance_test.create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)

{'orderId': 3521548609,
 'symbol': 'BTCUSDT',
 'status': 'NEW',
 'clientOrderId': 'UyYlU7GWzsuEU2ETGeE4yb',
 'price': '0.00',
 'avgPrice': '0.00',
 'origQty': '0.005',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'BUY',
 'positionSide': 'BOTH',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'MARKET',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1699730954056}